In [193]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import yfinance as yf
import plotly.graph_objs as go
from finta import TA
import holoviews as hv



In [194]:
# Create a function to pull in data using Yahoo finance API
def asset_df(*x):
    
    data = yf.download(tickers=[*x], period = '90d', interval = '1d')
    df = data['Close'].dropna()
    return df

In [200]:
# Dow = ^DJI
# SP500 = ^GSPC
# 30-Year TSY = ^TYX
# Crypto pairs 'CRYPTOTICKER-USD'
# Gold 'GC=F'
# Oil 'CL=F'

df = asset_df('BTC-USD','^DJI','^GSPC','^TYX','GC=F','CL=F')
df

[*********************100%***********************]  6 of 6 completed


,BTC-USD,CL=F,GC=F,^DJI,^GSPC,^TYX
Date,,,,,,
2021-09-15,48176.347656,72.610001,1792.400024,34814.390625,4480.700195,1.869
2021-09-16,47783.359375,72.610001,1754.599976,34751.320312,4473.750000,1.881
2021-09-17,47267.519531,71.970001,1749.400024,34584.878906,4432.990234,1.909
2021-09-20,42843.800781,70.290001,1761.800049,33970.468750,4357.729980,1.846
2021-09-21,40693.675781,70.559998,1776.000000,33919.839844,4354.189941,1.857
...,...,...,...,...,...,...
2021-12-07,50700.085938,72.050003,1782.599976,35719.429688,4686.750000,1.796
2021-12-08,50504.796875,72.360001,1783.400024,35754.750000,4701.209961,1.875
2021-12-09,47672.121094,70.940002,1774.599976,35754.691406,4667.450195,1.866


In [201]:
# Create function to run correlation matrix on any asset
def asset_corr(df):
    pct_chg_df = df.pct_change()
    df_corr = pct_chg_df.corr()
    df_corr_plot = df_corr.hvplot.heatmap()
    return df_corr_plot

In [202]:
asset_corr(df)

:HeatMap   [columns,index]   (value)

In [203]:
# Convert df to % change for analysis
pct_chg_df = df.pct_change()
pct_chg_df = pct_chg_df.dropna()

# RF Model

In [206]:
# Define features set
# Manually Change the first arugument to whatever you want to predict
X = pct_chg_df.copy()
X.drop("BTC-USD", axis=1, inplace=True)
X.head()

,CL=F,GC=F,^DJI,^GSPC,^TYX
Date,,,,,
2021-09-16,0.000000,-0.021089,-0.001812,-0.001551,0.006421
2021-09-17,-0.008814,-0.002964,-0.004789,-0.009111,0.014886
2021-09-20,-0.023343,0.007088,-0.017765,-0.016977,-0.033002
2021-09-21,0.003841,0.008060,-0.001490,-0.000812,0.005959
2021-09-22,0.023668,0.000394,0.009979,0.009520,-0.003231


In [207]:
# Define target vector
y = pct_chg_df["BTC-USD"].values.reshape(-1, 1)
y[:5]

array([[-0.00815729],
       [-0.01079539],
       [-0.09358898],
       [-0.05018521],
       [ 0.07079311]])

In [208]:
# Create function for RF Model

def rf_rscore(X,y):
    
    #split data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    #initiate, fit, and train rf_model
    rf_model = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_model = rf_model.fit(X_train, y_train)
    
    #make predictions for target using X_test
    predictions = rf_model.predict(X_test)
    
    return rf_model.score(X_test, y_test)

In [209]:
rf_rscore(X,y)

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


-0.03382379950812786

In [111]:
# # List the top 10 most important features
# importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
# importances_sorted[:10]

# Algo Trader

In [175]:
# def boll_bands(ticker):
#     # Create a new clean copy of the signals_df DataFrame
#     bb_signals_df = yf.download(tickers = ticker, period = '90d', interval = '1d')
    
#     # Determine the Bollinger Bands for the Dataset
#     bbands_df = TA.BBANDS(bb_signals_df)
    
#     # Concatenate the Bollinger Bands to the DataFrame
#     bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)
    
#     # Visualize close price for the investment
#     security_close = bb_signals_df[["Close"]].hvplot(
#         line_color='lightgray',
#         ylabel='Price in $',
#         width=1000,
#         height=400
#     )

#     bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
#         line_color='purple',
#         ylabel='Price in $',
#         width=1000,
#         height=400
#     )


#     bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
#         line_color='orange',
#         ylabel='Price in $',
#         width=1000,
#         height=400
#     )

#     bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
#         line_color='blue',
#         ylabel='Price in $',
#         width=1000,
#         height=400
#     )

#     # Overlay plots
#     bbands_plot = security_close * bb_upper * bb_middle * bb_lower
#     return bbands_plot

In [176]:
# boll_bands('ETH-USD')

[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (BB_UPPER)
   .Curve.III :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [Date]   (BB_LOWER)

# Create Moving Averages

In [114]:
# # Create a signals_df DataFrame that is a copy of the data Dataframe
# signals_df = yf.download(tickers = 'BTC-USD', period = '250d', interval = '1d')

# # Set the short window and long windows
# short_window = 25
# long_window = 100

# # Add the SMA technical indicators for the short and long windows
# signals_df["Short"] = TA.SMA(signals_df, short_window)
# signals_df["Long"] = TA.SMA(signals_df, long_window)

# # Determine the Bollinger Bands for the Dataset
# bbands_df = TA.BBANDS(signals_df)
    
# # Concatenate the Bollinger Bands to the DataFrame
# all_signals_df = pd.concat([signals_df, bbands_df], axis=1)

# # Review the DataFrame
# all_signals_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Short,Long,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2021-04-07,58186.507812,58731.144531,55604.023438,56048.937500,56048.937500,75645303584,NaN,NaN,NaN,NaN,NaN
2021-04-08,56099.914062,58338.738281,55879.085938,58323.953125,58323.953125,53053855641,NaN,NaN,NaN,NaN,NaN
2021-04-09,58326.562500,58937.046875,57807.863281,58245.003906,58245.003906,46655208546,NaN,NaN,NaN,NaN,NaN
2021-04-10,58253.777344,61276.664062,58038.707031,59793.234375,59793.234375,58238470525,NaN,NaN,NaN,NaN,NaN
2021-04-11,59846.230469,60790.554688,59289.796875,60204.964844,60204.964844,46280252580,NaN,NaN,NaN,NaN,NaN


In [115]:
# # Set the Signal column
# all_signals_df["Signal"] = 0.0

# # Generate the trading signal 1 or 0,
# # where 1 is when the Short window is greater than (or crosses over) the Long Window
# # where 0 is when the Short window is under the Long window
# all_signals_df["Signal"][short_window:] = np.where(
#     all_signals_df["Short"][short_window:] > all_signals_df["Long"][short_window:], 1.0, 0.0)

# # Calculate the points in time at which a position should be taken, 1 or -1
# all_signals_df["Entry/Exit"] = all_signals_df["Signal"].diff()

# # Review the DataFrame
# all_signals_df.head()

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,High,Low,Close,Adj Close,Volume,Short,Long,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal,Entry/Exit
Date,,,,,,,,,,,,,
2021-04-07,58186.507812,58731.144531,55604.023438,56048.937500,56048.937500,75645303584,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2021-04-08,56099.914062,58338.738281,55879.085938,58323.953125,58323.953125,53053855641,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-09,58326.562500,58937.046875,57807.863281,58245.003906,58245.003906,46655208546,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-10,58253.777344,61276.664062,58038.707031,59793.234375,59793.234375,58238470525,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-11,59846.230469,60790.554688,59289.796875,60204.964844,60204.964844,46280252580,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [117]:
# # Visualize entry position relative to close price
# entry = all_signals_df[all_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
#     color='purple',
#     marker='^',
#     size=200,
#     legend=False,
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# # Visualize exit position relative to close price
# exit = all_signals_df[all_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
#     color='orange',
#     marker='v',
#     size=200,
#     legend=False,
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# # Visualize close price for the investment
# security_close = all_signals_df[["Close"]].hvplot(
#     line_color='lightgray',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# # Visualize moving averages
# moving_avgs = all_signals_df[["Short", "Long"]].hvplot(
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# # Visualize close price for the investment
# security_close = all_signals_df[["Close"]].hvplot(
#     line_color='lightgray',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# bb_upper = all_signals_df[["BB_UPPER"]].hvplot(
#     line_color='purple',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )


# bb_middle = all_signals_df[["BB_MIDDLE"]].hvplot(
#     line_color='orange',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )

# bb_lower = all_signals_df[["BB_LOWER"]].hvplot(
#     line_color='blue',
#     ylabel='Price in $',
#     width=1000,
#     height=400
# )
    
# # Overlay plots
# entry_exit_plot = security_close * moving_avgs * entry * exit * bb_upper * bb_middle * bb_lower
# entry_exit_plot

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)

# Create function for algo trader that works on any ticker

In [218]:
def algo_trader(ticker):
    
    # Create a signals_df DataFrame that is a copy of the data Dataframe
    signals_df = yf.download(tickers = ticker, period = '250d', interval = '1d')

    # Set the short window and long windows
    short_window = 25
    long_window = 100

    # Add the SMA technical indicators for the short and long windows
    signals_df["Short"] = TA.SMA(signals_df, short_window)
    signals_df["Long"] = TA.SMA(signals_df, long_window)

    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(signals_df)

    # Concatenate the Bollinger Bands to the DataFrame
    all_signals_df = pd.concat([signals_df, bbands_df], axis=1)

    # Set the Signal column
    all_signals_df["Signal"] = 0.0

    # Generate the trading signal 1 or 0,
    # where 1 is when the Short window is greater than (or crosses over) the Long Window
    # where 0 is when the Short window is under the Long window
    all_signals_df["Signal"][short_window:] = np.where(
        all_signals_df["Short"][short_window:] > all_signals_df["Long"][short_window:], 1.0, 0.0)

    # Calculate the points in time at which a position should be taken, 1 or -1
    all_signals_df["Entry/Exit"] = all_signals_df["Signal"].diff()

    # Visualize entry position relative to close price
    entry = all_signals_df[all_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize exit position relative to close price
    exit = all_signals_df[all_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize close price for the investment
    security_close = all_signals_df[["Close"]].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize moving averages
    moving_avgs = all_signals_df[["Short", "Long"]].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )


    # Visualize Bollinger Bands
    bb_upper = all_signals_df[["BB_UPPER"]].hvplot(
        line_color='purple',
        ylabel='Price in $',
        width=1000,
        height=400
    )


    bb_middle = all_signals_df[["BB_MIDDLE"]].hvplot(
        line_color='orange',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    bb_lower = all_signals_df[["BB_LOWER"]].hvplot(
        line_color='blue',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Overlay plots
    entry_exit_plot = security_close * moving_avgs * entry * exit * bb_upper * bb_middle * bb_lower
    return entry_exit_plot

In [224]:
algo_trader('ETH-USD')

[*********************100%***********************]  1 of 1 completed


/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)

# Work in progress

In [225]:
# Add the SMA technical indicators for the short and long windows
rsi = TA.RSI(signals_df,period=14)


In [226]:
signals_df

,Open,High,Low,Close,Adj Close,Volume,Short,Long
Date,,,,,,,,
2021-04-07,58186.507812,58731.144531,55604.023438,56048.937500,56048.937500,75645303584,NaN,NaN
2021-04-08,56099.914062,58338.738281,55879.085938,58323.953125,58323.953125,53053855641,NaN,NaN
2021-04-09,58326.562500,58937.046875,57807.863281,58245.003906,58245.003906,46655208546,NaN,NaN
2021-04-10,58253.777344,61276.664062,58038.707031,59793.234375,59793.234375,58238470525,NaN,NaN
2021-04-11,59846.230469,60790.554688,59289.796875,60204.964844,60204.964844,46280252580,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-12-08,50667.648438,51171.375000,48765.988281,50504.796875,50504.796875,28479699446,56342.602656,54569.385313
2021-12-09,50450.082031,50797.164062,47358.351562,47672.121094,47672.121094,29603577251,55630.813906,54574.439648
2021-12-10,47642.144531,50015.253906,47023.699219,47243.304688,47243.304688,30966005122,54978.231250,54558.402422


In [221]:
# Visualize RSI for the investment
rsi_plot = rsi.hvplot(
    line_color='lightgray',
    ylabel='RSI',
    width=1000,
    height=400,
    ylim=[0,100]

)

final_rsi_plot = rsi_plot * hv.HLine(70).opts(color='red') *hv.HLine(30).opts(color='lightgreen')


In [222]:
final_rsi_plot

:Overlay
   .Curve.A_14_period_RSI :Curve   [Date]   (14 period RSI)
   .HLine.I               :HLine   [x,y]
   .HLine.II              :HLine   [x,y]